**Tran Van Luc**

**20521587**

**Lab 3 : Dữ liệu XML**

### Import library

In [15]:
from lxml import etree as ET
import pandas as pd

In [2]:
ELEMENTS = ['article', 'inproceedings', 'proceedings', 'book', 'incollection', 'phdthesis', 'mastersthesis']

FEATURES = ['author', 'title', 'school', 'year', 'pages', 'ee', 'publisher', 'booktitle', 'address', 'journal', 'volume', 
            'number', 'month', 'url', 'cdrom', 'editor', 'cite', 'note', 'crossref', 'isbn', 'series', 'chapter', 'publnr']

In [3]:
def handle_coincided(tag, text, value):
    # Hàm này sẽ tìm tất cả các vị trí của value trong tag, lấy các giá trị tương ứng trong text
    # sau đó nối chúng thành một chuỗi phân tách bởi dấu phẩy, và gán chuỗi này cho vị trí đầu tiên của value trong text.
    # sau đó loại bỏ tất cả các vị trí khác của value trong tag và text 
    tag_cp = tag.copy()
    text_cp = text.copy()
    count = tag_cp.count(value)
    index = tag_cp.index(value)
    list_connect = []
    for i in range(index, index + count):
        list_connect.append(text_cp[i])
        list_connect = ', '.join(list_connect)
    for i in range(count - 1):
        tag_cp.remove(value)
    for i in range(index, index + count):
        text_cp.remove(text[i])
    text_cp.insert(index, list_connect)
    return tag_cp, text_cp

### Read file XML 

In [8]:
def Read_XML(value, num_features):
    # Sủ dụng ET.iterparse() vì phương thức này được sử dụng để đọc một tài liệu XML lớn và không giữ nó hoàn toàn trong bộ nhớ, giúp tránh tình trạng hết bộ nhớ khi đọc tệp XML lớn
    # tree = ET.iterparse('./dblp.xml', dtd_validation=True, load_dtd=True)
    tree = ET.iterparse('./dblp.xml', dtd_validation=True, load_dtd=True)
    tags = []
    texts = []
    count = 0
    df = pd.DataFrame(columns = FEATURES)
    # print(df)
    for _, element in tree:
        if(element.tag in FEATURES):
            # có thẻ nằm trong danh sách các thuộc tính của FEATURES thì nó sẽ được thêm vào danh sách tags, texts
            tags.append(element.tag)
            texts.append(element.text)
        elif(element.tag in ELEMENTS):
            if(element.tag == value):
                try:
                    tags.append('ELM')
                    texts.append(element.tag)
                    # xử lý các phần tử trùng lặp bằng cách gọi hàm handle_coincided
                    unique = list(set(tags))
                    for i in unique:
                        if(tags.count(i) > 1):
                            tags, texts = handle_coincided(tags, texts, i)
                    # tạo dữ liệu bằng cách duyệt qua danh sách FEATURES 
                    # và thêm dữ liệu tương ứng từ danh sách texts vào mảng dữ liệu nếu thẻ đã được xét có tên tương ứng trong danh sách FEATURES
                    # ngược lại sẽ thêm giá trị None
                    arr = []
                    for i in range(len(FEATURES)):
                        if(FEATURES[i] in tags):
                            index = tags.index(FEATURES[i])
                            arr.append(texts[index])
                        else:
                            arr.append(None)
                    # tạo DataFrame từ mảng dữ liệu trên
                    keys = FEATURES
                    items = arr
                    data = dict(zip(keys, items))
                    df = pd.concat([df, pd.DataFrame.from_records([data])])
                    count = count + 1

                except:
                    pass
                if(count == num_features): 
                    break
            tags = []
            texts = []
        # giải phóng bộ nhớ và tránh tình trạng tràn bộ nhớ.
        element.clear()
        while element.getprevious() is not None:
            del element.getparent()[0]  
    return df

In [9]:
df_final = Read_XML('mastersthesis', 24)

In [12]:
df_final = df_final.dropna(axis=1, how='all')

In [13]:
df_final

author                                              title  \
0             Tatu Ylönen                         Shadow Paging Is Feasible.   
0           Peter Van Roy                     A Prolog Compiler for the PLM.   
0        Martin Monperrus                        Manifolds: beyond locality.   
0             Leo Liberti  Fondamenti algebrici degli automi cellulari in...   
0            Salim Perchy  Multimedia Interaction with NTCC. (Interacción...   
0  Christian Schulte 0001  Entwurf und Implementierung eines übersetzende...   
0               Liang Dai  Online Controlled Experiment Design: Trade-off...   
0         Satyan R. Coorg  Partitioning non-strict languages for multi-th...   
0           Kurt P. Brown  PRPL: A Database Workload Specification Langua...   
0                Rita Ley  Der Einfluss kleiner naturnaher Retentionsmaßn...   
0    Oliver Hoffmann 0002  Regelbasierte Extraktion und asymmetrische Fus...   
0             Tolga Yurek     Efficient View Maintenance at Data Warehouses.   
0         Stephan Vollmer  Portierung des DBLP-Systems auf ein relational...   

                                              school  year  \
0  Helsinki University of Technology, Department ...  1994   
0               University of California at Berkeley  1984   
0      University of Technology of Compiègne, France  2004   
0                         University of Turin, Italy  1997   
0  Pontifical Xavierian University, Bogotá, Colombia  2013   
0  Universität Karlsruhe, Institut für Logik, Kom...  1991   
0          University of California, Santa Cruz, USA  2014   
0  Massachusetts Institute of Technology, Cambrid...  1994   
0                    University of Wisconsin-Madison  1992   
0  Diplomarbeit, Universität Trier, FB VI, Physis...  2006   
0                                University of Trier  2009   
0  University of California at Santa Barbara, Dep...  1997   
0  Diplomarbeit, Universität Trier, FB IV, Inform...  2006   

                                                  ee  \
0                                               None   
0                                               None   
0      https://tel.archives-ouvertes.fr/hal-01253225   
0      https://tel.archives-ouvertes.fr/hal-00163561   
0      https://tel.archives-ouvertes.fr/tel-01257184   
0                                               None   
0      https://www.escholarship.org/uc/item/1hm5t2z6   
0                https://hdl.handle.net/1721.1/32607   
0                                               None   
0  http://dblp.uni-trier.de/papers/DiplomarbeitRi...   
0  http://dblp.uni-trier.de/papers/DiplomarbeitOl...   
0                                               None   
0  http://dbis.uni-trier.de/Diplomanden/Vollmer/v...   

                                                note  
0                                               None  
0                                               None  
0                                               None  
0                                               None  
0                                               None  
0                                               None  
0                                               None  
0        ndltd.org (oai:dspace.mit.edu:1721.1/32607)  
0                                               None  
0                                               None  
0  Diplomarbeit, Universität Trier, FB IV, DBIS/DBLP  
0                                               None  
0                                               None

In [14]:
for i in ELEMENTS:
    df_crawl = Read_XML(i, 100000)
    df_crawl.to_csv(i + '_data.csv', index=False)

In [11]:
df_final.to_csv('mastersthesis_data.csv', index = False)